In [1]:
import os
import csv
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from collections import defaultdict

In [2]:
# pr_file_path="/Users/janeliu/Desktop/2019fall/csp584/final/Recommander-for-Social-Places/web"
pr_file_path=os.getcwd()
print(pr_file_path)
path_upper = os.path.abspath(os.path.dirname(os.getcwd()))
#load mongodb's path
f = open(os.path.join(path_upper,"config.txt"))
for line in f:
    info = line.split("=")
    if info[0] == "MONGODB_BIN":
        mongo_db = info[1].replace("\n","")
        print(mongo_db)
        os.chdir(mongo_db)
os.system(r'./mongoexport --db=PlaceReviews --collection=myReviews --type=csv --fields=userName,businessId,reviewrating > '+os.path.join(pr_file_path,'mongodata_train.csv'))

/Users/janeliu/eclipse-workspace/apache-tomcat-7.0.34/webapps/Recommender_for_Social_Places/data
/usr/local/mongodb/bin/
./mongoexport --db=PlaceReviews --collection=myReviews --type=csv --fields=userName,businessId,reviewrating > "/Users/janeliu/eclipse-workspace/apache-tomcat-7.0.34/webapps/Recommender_for_Social_Places/data/mongodata_train.csv"


512

In [3]:
with open(os.path.join(pr_file_path,"mongodata_train.csv"), "r") as f:
    reader = csv.DictReader(f, delimiter=',')
    with open(os.path.join(pr_file_path,"mongodata_test.csv"), "w",newline='')as f_out:
        writer = csv.DictWriter(f_out, fieldnames=reader.fieldnames, delimiter=",")
        for row in reader:
            writer.writerow(row)
file_path = os.path.expanduser(os.path.join(pr_file_path,'mongodata_test.csv'))

In [4]:
reader = Reader(line_format='user item rating', sep=',')

#data = Dataset.load_from_file(file_path, reader=reader)

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_from_file(file_path, reader=reader)
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=3)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    
out = open(os.path.join(pr_file_path,'output.csv'), 'w',newline='')
output=csv.writer(out)

for uid, user_ratings in top_n.items():
    output.writerow([uid, [iid for (iid, _) in user_ratings]])
    
out.close()


user1 ['qD3-5yqSyPiz-VpJtPsnkg', 'xAZx3d8rEBzKVtOI5Vf5xA', 'U5tVK8ChZPP_72ldZPmqAA']
user2 ['-l_EorQgXO9VImb5AvV4nQ', 'yDCyAklnWxnzV675_iweVQ', 'lYKn_TKwzudoIYplL2tGDg']
user3 ['-l_EorQgXO9VImb5AvV4nQ', 'TFmh4YqjovxtxQ8K2k3ing', 'yDCyAklnWxnzV675_iweVQ']
user4 ['xAZx3d8rEBzKVtOI5Vf5xA', 'yDCyAklnWxnzV675_iweVQ', 'DXwSYgiXqIVNdO9dazel6w']
user5 ['-l_EorQgXO9VImb5AvV4nQ', 'qD3-5yqSyPiz-VpJtPsnkg', 'DXwSYgiXqIVNdO9dazel6w']
user6 ['qD3-5yqSyPiz-VpJtPsnkg', '-l_EorQgXO9VImb5AvV4nQ', 't_0jtlPiS9YK86LjhTTleg']
user7 ['yDCyAklnWxnzV675_iweVQ', 'DXwSYgiXqIVNdO9dazel6w', 'zIXZtgiir9CVF-iZeeAvaQ']
user8 ['yDCyAklnWxnzV675_iweVQ', 'DXwSYgiXqIVNdO9dazel6w', 'o0DaPvRb4IWH7hUWVCnoGw']
user9 ['yDCyAklnWxnzV675_iweVQ', 'qD3-5yqSyPiz-VpJtPsnkg', 'DXwSYgiXqIVNdO9dazel6w']
